In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

from wave_radar import s2wave2uv_br


In [10]:
# filein = '../../../DATA/49HGE nativem.nc'

basedir = '../RPSLinkage/DATA/20240127'
# basedir = '../RPSLinkage/DATA/20240421'

tile = '49KGR'

filein = f'{basedir}/{tile} 10m.nc'


ds_in = xr.open_dataset(filein)
ds_in

<xarray.Dataset>
Dimensions:     (LAT: 10980, LON: 10980, RGB: 3)
Coordinates:
  * LAT         (LAT) float64 -21.68 -21.68 -21.68 ... -22.68 -22.68 -22.68
  * LON         (LON) float64 112.9 112.9 112.9 112.9 ... 114.0 114.0 114.0
  * RGB         (RGB) int64 0 1 2
Data variables:
    B02         (LAT, LON) uint16 ...
    B04         (LAT, LON) uint16 ...
    B02_DETFOO  (LAT, LON) uint8 ...
    B04_DETFOO  (LAT, LON) uint8 ...
    x           (LON) float64 ...
    y           (LAT) float64 ...
    X           (LAT, LON) float64 ...
    Y           (LAT, LON) float64 ...
Attributes: (12/14)
    chunk:                     49KGR
    resolution_m:              10
    sensing_date_unix:         20240127T024311
    z1:                        49
    z2:                        K
    band_timing_nbar_band_02:  4.7
    ...                        ...
    band_timing_nbar_band_04:  5.7
    band_timing_nbar_band_05:  6.0
    band_timing_nbar_band_06:  6.2
    band_timing_nbar_band_07:  6.5
    band_timing_nbar_band_12:  6.8
    export_filename:           /mnt/c/Users/00071913/OneDrive - The Universit...

In [11]:
# ds_in['B02_DETFOO'].plot.imshow()
# ds_in['B04'].plot.imshow(vmax=5000, cmap='gray')
# plt.grid(True)

In [12]:
#######################
# Processing Parameters
#######################

# fileout = f'../../../DATA/{tile}_nativem_uv.nc'
# granule_n = int(256)
# nperseg = int(granule_n/4)
# noverlap = int(granule_n/4)
# allow_bias = False

# fileout = f'{basedir}/{tile}_nativem_uv_128.nc'
# granule_n = int(128)
# nperseg = int(granule_n/2)
# noverlap = int(granule_n/2)
# allow_bias = False

# fileout = f'{basedir}/{tile}_10m_uv_bias.nc'
# granule_n = int(256)
# nperseg = int(granule_n/4)
# noverlap = int(granule_n/4)
# allow_bias = True

fileout = f'{basedir}/{tile}_10m_uv_bias_512.nc'
granule_n = int(512)
nperseg = int(granule_n/8)
noverlap = int(granule_n/8)
allow_bias = True

#fileout = None # Don't save output

dt = np.nan
dx = 10
windowfunc = None
# coherence_threshold = 0.9
coherence_threshold = 0.8 
#coherence_threshold = 0.3 
# coherence_threshold = 0.1
minpoints = 30
Lmin = 60
Lmax = 300
F0max = 20



In [ ]:
%%time
timelags = {
    'B02':np.array([np.nan, 4.7, 2.6,4.7,2.6,4.7,2.6,4.7,2.6,4.7,2.6,4.7,2.6 ]),
    'B04':np.array([np.nan, 5.7, 1.6,5.7,1.6,5.7,1.6,5.7,1.6,5.7,1.6,5.7,1.6 ]),
}

print('Dealing with time lags...')
times = []
times.append(timelags['B02'][ds_in['B02_DETFOO']])
times.append(timelags['B04'][ds_in['B04_DETFOO']])
print('    ...done')

if False:
    plt.figure()
    mb = plt.imshow(ds_in['B02_DETFOO'])
    plt.title('Detector number for Beam 2')
    plt.colorbar(mb)
    plt.show()

nx = ds_in.LON.shape[0]
ny = ds_in.LAT.shape[0]

data = np.zeros((2,ny,nx))

data
data[0,...] = ds_in['B02'].values
data[1,...] = ds_in['B04'].values
    
granule = None
granule

# u,v,err,bias,N,X,Y,eigvalgrd,eigvecgrd,sigma_e2grd,granules 
output =  s2wave2uv_br.process_scene(data, times, ds_in.x.values, ds_in.y.values, dx, 
                                        granule_n = granule_n,
                                        nperseg = nperseg,
                                        noverlap = noverlap,
                                        coherence_threshold = coherence_threshold,
                                        minpoints = minpoints,
                                        Lmin=Lmin,
                                        Lmax=Lmax,
                                        F0max=F0max,
                                        windowfunc = windowfunc,
                                        allow_bias=allow_bias,
                                        output_full=True)


Dealing with time lags...
    ...done
Allowing bias


In [ ]:
dsout = s2wave2uv_br.output_to_xr(ds_in, filein, output)\
    .drop_vars(['granule_0_grd',  'granule_1_grd'])

dsout

In [ ]:
comp = dict(zlib=True, complevel=5)
encoding = {var: comp for var in dsout.data_vars}
dsout.to_netcdf(fileout, encoding=encoding)
if fileout is not None:
    dsout.to_netcdf(fileout, encoding=encoding)
    print(fileout)

In [ ]:
plt.figure()
plt.quiver(dsout['X'], dsout['Y'], dsout['u'], dsout['v'])

In [ ]:
dsout['bias'].plot()